In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [5]:
# Step 1: Load data
dataframe = pd.read_csv('data.csv')
# remove unnecessary features
dataframe = dataframe.drop(['index', 'price', 'sq_price'], axis=1)
dataframe = dataframe[:10]
dataframe

,area,bathrooms
0,2104.0,3.0
1,1600.0,3.0
2,2400.0,3.0
3,1416.0,2.0
4,3000.0,4.0
5,1985.0,4.0
6,1534.0,3.0
7,1427.0,3.0
8,1380.0,3.0
9,1494.0,3.0


In [6]:
# Step 2 - Adding labels
# 1 - good buy
# 0 - bad buy
dataframe.loc[:, 'y1'] = [1,1,1,0,0,1,0,1,1,1]

In [11]:
# Cast the true and false to int
dataframe.loc[:, 'y2'] = (dataframe['y1'] == 0).astype(int)
dataframe

,area,bathrooms,y1,y2
0,2104.0,3.0,1,0
1,1600.0,3.0,1,0
2,2400.0,3.0,1,0
3,1416.0,2.0,0,1
4,3000.0,4.0,0,1
5,1985.0,4.0,1,0
6,1534.0,3.0,0,1
7,1427.0,3.0,1,0
8,1380.0,3.0,1,0
9,1494.0,3.0,1,0


In [18]:
# Step 3 - prepare data for tensorflow
# tensors - generic version of vectors and matrices
# vector - list of numbers (1D tensor)
# matrix - list of list of numbers (2D tensor)
# list of list of list of numbers (3D tensor)

# placeholder for input tensor
inputX = dataframe.loc[:, ['area', 'bathrooms']].values
inputY = dataframe.loc[:, ['y1', 'y2']].values

In [19]:
# Step 4 - write out our hyperparameters
learning_rate = 0.000001
training_epochs = 2000
display_step = 50
n_samples = inputY.size

In [31]:
# Step 5 - Create computation graph / neural network

# for feature input tensors; None -> any number of examples
x = tf.placeholder(tf.float32, [None, 2])

# create weights
# 2 x 2 float matrix that we'll keep updating through the training process
# variables in tf hold and update parameters
# in memory buffers containing tensors
W = tf.Variable(tf.zeros([2, 2]))

# biases
b = tf.Variable(tf.zeros([2]))

# multiply weights by inputs
y_values = tf.add(tf.matmul(x,  W), b)

# apply softmax - activation function
y = tf.nn.softmax(y_values)

# feed in a matrix of labels
y_ = tf.placeholder(tf.float32, [None, 2])

In [32]:
# Step 6 - perform training
# Create cost function: mean squared error
# reduce sum computes the sum of elements across dimensions of a tensor

cost = tf.reduce_sum(tf.pow(y_ - y, 2)) / (2 * n_samples)

# Minimize the cost using gradient descent

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [33]:
# Initialize variables and tensorflow session
init= tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [35]:
# training loop
for i in range(training_epochs):
    sess.run(optimizer, feed_dict={x: inputX, y_: inputY})
    if i % display_step == 0:
        cc = sess.run(cost, feed_dict={x: inputX, y_:inputY})
        print('Training step: ', '%04d' % (i), "cost=", "{:.9f}".format(cc))

print("Optimization Finished!")
training_cost = sess.run(cost, feed_dict={x: inputX, y_:inputY})
print("Training cost=", training_cost, "W=", sess.run(W), "b=", sess.run(b))

Training step:  0000 cost= 0.114958666
Training step:  0050 cost= 0.109539948
Training step:  0100 cost= 0.109539881
Training step:  0150 cost= 0.109539807
Training step:  0200 cost= 0.109539740
Training step:  0250 cost= 0.109539665
Training step:  0300 cost= 0.109539606
Training step:  0350 cost= 0.109539531
Training step:  0400 cost= 0.109539464
Training step:  0450 cost= 0.109539405
Training step:  0500 cost= 0.109539330
Training step:  0550 cost= 0.109539263
Training step:  0600 cost= 0.109539188
Training step:  0650 cost= 0.109539129
Training step:  0700 cost= 0.109539054
Training step:  0750 cost= 0.109538995
Training step:  0800 cost= 0.109538913
Training step:  0850 cost= 0.109538853
Training step:  0900 cost= 0.109538786
Training step:  0950 cost= 0.109538712
Training step:  1000 cost= 0.109538652
Training step:  1050 cost= 0.109538570
Training step:  1100 cost= 0.109538510
Training step:  1150 cost= 0.109538451
Training step:  1200 cost= 0.109538391
Training step:  1250 cost

In [36]:
sess.run(y, feed_dict={x: inputX})

array([[0.7112522 , 0.2887478 ],
       [0.66498977, 0.33501023],
       [0.73657656, 0.26342347],
       [0.6471879 , 0.3528121 ],
       [0.78335613, 0.2166439 ],
       [0.70069474, 0.29930523],
       [0.6586633 , 0.34133676],
       [0.6482863 , 0.35171372],
       [0.6436828 , 0.35631716],
       [0.65480113, 0.3451989 ]], dtype=float32)